This notebook is for sorting MNIST dataset.

The model used in this notebook is a simple fully connected neural network (FCN) with one hidden layer. The loss function used is cross-entropy loss and the optimizer used is SGD with momentum. The learning rate is set to 0.01 and the momentum is set to 0.9. 

In [ ]:
# import MNIST dataset
import torch
from torchvision import datasets, transforms

# Define transforms to apply to the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Load the dataset
trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('MNIST_data/', download=True, train=False, transform=transform)

# MPS加速支持
device_mps = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Define the data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the model
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # ？
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 784) # 作用是将输入的二维图像数据展平为一维数据
        x = nn.functional.relu(self.fc1(x)) # fc1是输入层，relu是激活函数
        x = nn.functional.relu(self.fc2(x)) # fc2是隐藏层，relu是激活函数
        x = self.fc3(x) # fc3是输出层
        return x

model = Net().to(device_mps)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # 优化器type SGD

# Train the model
for epoch in range(10):
    running_loss = 0.0 # 记录每一个epoch的loss
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device_mps), labels.to(device_mps)

        optimizer.zero_grad() # 梯度清零

        outputs = model(inputs) # 前向传播
        loss = criterion(outputs, labels) # 计算loss
        loss.backward() # 反向传播
        optimizer.step() # 更新参数

        running_loss += loss.item() # 累计loss
        if i % 2000 == 1999: # 每2000个batch打印一次loss
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    correct = 0      
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device_mps), labels.to(device_mps)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # 预测值
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 计算正确的个数

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 88 %
Accuracy of the network on the 10000 test images: 90 %
Accuracy of the network on the 10000 test images: 91 %
Accuracy of the network on the 10000 test images: 92 %
Accuracy of the network on the 10000 test images: 93 %
Accuracy of the network on the 10000 test images: 93 %
Accuracy of the network on the 10000 test images: 93 %
Accuracy of the network on the 10000 test images: 94 %
Accuracy of the network on the 10000 test images: 94 %
Accuracy of the network on the 10000 test images: 95 %


As the MNIST contains images, we can use convolutional neural networks (CNNs) to classify the images.

Here is the instructions from ChatGPT:

是的，MNIST 数据集非常适合使用卷积神经网络（CNN）进行处理。CNN 在图像识别任务中表现出色，能够有效地捕捉图像的空间特征，从而提高分类准确率。

**使用 CNN 处理 MNIST 数据集的步骤：**

1. **加载 MNIST 数据集：** MNIST 是一个手写数字数据集，包含 60,000 张训练图像和 10,000 张测试图像，每张图像为 28x28 像素的灰度图像。

2. **数据预处理：** 将图像数据归一化到 [0,1] 区间，并将标签进行独热编码。

3. **定义 CNN 模型：** 通常包括以下层次结构：
   - **卷积层：** 用于提取图像的局部特征。
   - **池化层：** 用于下采样，减少特征图的尺寸。
   - **全连接层：** 用于将提取的特征映射到分类结果。

4. **编译模型：** 选择适当的损失函数（如交叉熵损失）和优化器（如 Adam）。

5. **训练模型：** 使用训练数据进行模型训练，并在验证集上监测性能。

6. **评估模型：** 在测试数据上评估模型的准确性。

通过上述步骤，使用 CNN 可以在 MNIST 数据集上实现高达 99% 以上的分类准确率。例如，有研究通过集成简单的 CNN 模型，在 MNIST 测试集上实现了 99.87% 的准确率。 ([arxiv.org](https://arxiv.org/abs/2008.10400?utm_source=chatgpt.com))

如果您对具体的实现细节感兴趣，可以参考以下资源，了解如何使用 PyTorch 或 TensorFlow 等深度学习框架来构建和训练 CNN 模型：

- [PyTorch 实现 CNN 识别手写数字（MNIST 数据集）](https://zhuanlan.zhihu.com/p/544161254)
- [TensorFlow 实现 CNN 处理 MNIST 数据集](https://developer.aliyun.com/article/604770)

这些资源提供了详细的代码示例和解释，有助于您更好地理解如何使用 CNN 处理 MNIST 数据集。 


In [ ]:
# 按照上面FCN方法中，加载MNIST数据集

# import MNIST dataset
import torch
from torchvision import datasets, transforms

# Define transforms to apply to the data
transform = transforms.Compose([transforms.ToTensor()]) # Normalize the data to [0, 1]

# Load the dataset
trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('MNIST_data/', download=True, train=False, transform=transform)

# MPS加速支持
device_mps = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Define the data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
